# NBA Games Learning 

This notebook takes datasets, tests a few models and outputs the results.

## Configuration

In [1]:
output_parsed_data_filename = '../input/parsed_data.csv'
output_test_data_filename = '../input/parsed_data_current_games.csv'

save_result_data = True
output_result_data_filename = '../output/result_current_season.csv'

# Processor

In [2]:
import json 
import pandas as pd
import numpy as np 
import geopy.distance

#package for flattening json in pandas df
from pandas.io.json import json_normalize

In [3]:
# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

## Load data from file system

In [4]:
games = pd.read_csv(output_parsed_data_filename)
games = games.drop(['Unnamed: 0'], axis=1)

/Users/jkentjnr/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
gamesTest = None

if output_test_data_filename is not None:
    gamesTest = pd.read_csv(output_test_data_filename)
    gamesTest = gamesTest.drop(['Unnamed: 0'], axis=1)

In [6]:
len(games)

14324

In [7]:
games.sample(5)

,id,location,scoreAway,scoreHome,scoreQuarters,season,statsAway.Ast,statsAway.Blk,statsAway.BlkAgainst,statsAway.DefReb,...,calcStatsHome.last5DefensiveRating,calcStatsAway.seasonTotalPossessions,calcStatsAway.last5TotalPossessions,calcStatsAway.seasonTotalPoints,calcStatsAway.last5TotalPoints,calcStatsAway.seasonOffensiveRating,calcStatsAway.last5OffensiveRating,calcStatsAway.seasonDefensiveRating,calcStatsAway.last5DefensiveRating,homeWin
1815,34529,AT&T Center,94,134,"[{'number': 1, 'scoreHome': 36, 'scoreAway': 2...",2016,23,3,5,22,...,122.58,3393.6,490.2,3980,607,117.28,123.83,112.68,109.75,True
13489,201501120BOS,NaN,100,108,"[{'number': 1, 'scoreHome': 22, 'scoreAway': 2...",2014,20,6,5,29,...,106.63,6400.6,450.4,6928,494,108.24,109.68,105.90,105.91,True
11637,201311010PHO,NaN,84,87,"[{'number': 1, 'scoreHome': 29, 'scoreAway': 2...",2013,20,1,10,31,...,106.79,5261.2,469.2,5687,494,108.09,105.29,103.93,93.56,True
2237,34952,Talking Stick Resort Arena,107,101,"[{'number': 1, 'scoreHome': 29, 'scoreAway': 2...",2016,22,11,11,42,...,104.64,3370.6,450.4,3957,560,117.40,124.33,113.69,117.01,False
2059,34773,Staples Center,97,96,"[{'number': 1, 'scoreHome': 23, 'scoreAway': 2...",2016,23,3,3,34,...,108.27,629.0,458.8,698,498,110.97,108.54,111.45,108.11,False


   ## Data Preperation

In [8]:
import math
def generateBestBetHome(row):
    ret = 0
    if (math.isnan(row["MatchbookHomeOdd"]) == False) & (row["MatchbookHomeOdd"] > ret):
        ret = row["MatchbookHomeOdd"]
    if (math.isnan(row["PinnacleHomeOdd"]) == False) & (row["PinnacleHomeOdd"] > ret):
        ret = row["PinnacleHomeOdd"]
    if (math.isnan(row["Bet365HomeOdd"]) == False) & (row["Bet365HomeOdd"] > ret):
        ret = row["Bet365HomeOdd"]

    return np.NaN if ret == 0 else ret
    
def generateBestBetAway(row):
    ret = 0
    if (math.isnan(row["MatchbookAwayOdd"]) == False) & (row["MatchbookAwayOdd"] > ret):
        ret = row["MatchbookAwayOdd"]
    if (math.isnan(row["PinnacleAwayOdd"]) == False) & (row["PinnacleAwayOdd"] > ret):
        ret = row["PinnacleAwayOdd"]
    if (math.isnan(row["Bet365AwayOdd"]) == False) & (row["Bet365AwayOdd"] > ret):
        ret = row["Bet365AwayOdd"]

    return np.NaN if ret == 0 else ret

games["homeBestOdds"] = games.apply(generateBestBetHome, axis=1)
games["awayBestOdds"] = games.apply(generateBestBetAway, axis=1)

if gamesTest is not None:
    gamesTest["homeBestOdds"] = gamesTest.apply(generateBestBetHome, axis=1)
    gamesTest["awayBestOdds"] = gamesTest.apply(generateBestBetAway, axis=1)

In [9]:
games = games.drop(["MatchbookHomeOdd", "PinnacleHomeOdd", "Bet365HomeOdd", "MatchbookAwayOdd", "PinnacleAwayOdd", "Bet365AwayOdd"], axis=1)

if gamesTest is not None:
    gamesTest = gamesTest.drop(["MatchbookHomeOdd", "PinnacleHomeOdd", "Bet365HomeOdd", "MatchbookAwayOdd", "PinnacleAwayOdd", "Bet365AwayOdd"], axis=1)

### Record the names of all stats - to be dropped later.

In [10]:
statsColumns = pd.Series(list(games.columns.values))
statsColumns = statsColumns.loc[statsColumns.str.startswith(('statsAway', 'statsHome'), na=False)]

In [11]:
# Replace any NaN with 0 in the stats
games[statsColumns] = games[statsColumns].fillna(value=0)

if gamesTest is not None:
    gamesTest[statsColumns] = gamesTest[statsColumns].fillna(value=0)

### Drop unrequired fields

In [12]:
games = games.drop(["time2", "day", "day2"], axis=1)

if gamesTest is not None:
    gamesTest = gamesTest.drop(["time2", "day", "day2"], axis=1)

In [13]:
games = games.drop(["teamAwayCode", "teamHomeCode"], axis=1)

if gamesTest is not None:
    gamesTest = gamesTest.drop(["teamAwayCode", "teamHomeCode"], axis=1)

In [14]:
games = games.drop(["location", "scoreQuarters", "time"], axis=1)

if gamesTest is not None:
    gamesTest = gamesTest.drop(["location", "scoreQuarters", "time"], axis=1)

In [15]:
games = games.drop(statsColumns, axis=1)

if gamesTest is not None:
    gamesTest = gamesTest.drop(statsColumns, axis=1)

In [16]:
# Get rid of all games with no odds
games = games[(pd.notnull(games['homeBestOdds']))]

if gamesTest is not None:
    gamesTest = gamesTest[(pd.notnull(gamesTest['homeBestOdds']))]

In [17]:
# Remove games scores ... not possible to predict the future, if you already know it :)
games = games.drop(["scoreAway", "scoreHome"], axis=1)

if gamesTest is not None:
    gamesTest = gamesTest.drop(["scoreAway", "scoreHome"], axis=1)

In [18]:
# Remove any games that have less than a 5 game history.
games = games[(games['totalGamesHome'] > 5) & (games['totalGamesAway'] > 5)]

if gamesTest is not None:
    gamesTest = gamesTest[(gamesTest['totalGamesHome'] > 5) & (gamesTest['totalGamesAway'] > 5)]

In [19]:
# Show any NaN
games[games.isnull().T.any().T]

,id,season,gamesPlayedHome,gamesPlayedAway,totalGamesHome,totalWinsHome,homeGamesHome,homeWinsHome,awayGamesHome,awayWinsHome,...,calcStatsAway.last5TotalPossessions,calcStatsAway.seasonTotalPoints,calcStatsAway.last5TotalPoints,calcStatsAway.seasonOffensiveRating,calcStatsAway.last5OffensiveRating,calcStatsAway.seasonDefensiveRating,calcStatsAway.last5DefensiveRating,homeWin,homeBestOdds,awayBestOdds


In [20]:
len(games)

6253

In [21]:
gamesTest.sample(5)

,id,season,gamesPlayedHome,gamesPlayedAway,totalGamesHome,totalWinsHome,homeGamesHome,homeWinsHome,awayGamesHome,awayWinsHome,...,statsAway.last5TotalPossessions,statsAway.seasonTotalPoints,statsAway.last5TotalPoints,statsAway.seasonOffensiveRating,statsAway.last5OffensiveRating,statsAway.seasonDefensiveRating,statsAway.last5DefensiveRating,homeWin,homeBestOdds,awayBestOdds
646,42726,2017,12,10,12,7,6,3,6,4,...,496.0,1056,549,109.45,110.69,113.50,106.45,True,2.20,1.81
169,42244,2017,43,41,40,22,18,13,22,9,...,454.4,4465,535,115.90,117.74,115.45,119.28,False,1.54,2.73
550,42630,2017,6,6,8,3,3,2,5,1,...,476.4,683,566,119.49,118.81,107.07,107.26,False,1.70,2.37
395,42473,2017,58,57,58,33,30,20,28,13,...,442.6,5935,494,117.02,111.61,114.63,111.61,False,3.38,1.43
231,42307,2017,47,46,45,26,21,19,24,7,...,449.6,4778,543,116.16,120.77,111.86,110.99,False,1.48,2.98


In [22]:
games.sample(7)

,id,season,gamesPlayedHome,gamesPlayedAway,totalGamesHome,totalWinsHome,homeGamesHome,homeWinsHome,awayGamesHome,awayWinsHome,...,calcStatsAway.last5TotalPossessions,calcStatsAway.seasonTotalPoints,calcStatsAway.last5TotalPoints,calcStatsAway.seasonOffensiveRating,calcStatsAway.last5OffensiveRating,calcStatsAway.seasonDefensiveRating,calcStatsAway.last5DefensiveRating,homeWin,homeBestOdds,awayBestOdds
1153,32330,2015,47,23,52,41,25,23,27,18,...,448.8,2188,556,120.72,123.89,116.42,118.32,False,1.03,22.00
11517,201304170NYK,2012,31,28,32,17,15,9,17,8,...,462.4,2282,513,104.85,110.94,103.98,99.48,True,1.31,4.05
13479,201501100MIN,2014,64,88,62,14,31,9,31,5,...,459.0,8776,501,106.86,109.15,102.75,105.23,False,1.50,2.80
7683,200910280OKC,2009,46,24,45,25,22,14,23,11,...,488.2,2077,533,107.41,109.18,105.75,107.33,True,1.36,3.55
2347,35062,2016,28,29,26,15,13,9,13,6,...,463.4,3284,570,121.88,123.00,119.06,121.92,True,1.52,2.80
119,31295,2015,51,52,48,22,25,16,23,6,...,456.8,4691,532,115.79,116.46,111.84,101.36,True,1.29,4.11
275,31451,2015,44,42,42,22,21,13,21,9,...,455.6,3643,502,116.61,110.18,114.02,104.04,False,1.19,5.50


## Randomise a training set

In [23]:
if gamesTest is None:
    msk = np.random.rand(len(games)) < 0.8
    msk

    train = games[msk]
    test = games[~msk]
else:
    train = games
    test = gamesTest
    
Y_train = train["homeWin"]
X_train = train.drop(["id","homeWin"], axis=1)

#
X_test  = test.drop(["id", "homeWin"], axis=1).copy()
X_train.shape, Y_train.shape, X_test.shape

((6253, 59), (6253,), (566, 59))

In [24]:
# Reset the index
test.index = range(len(test.index))

In [25]:
X_test.sample(5)

,season,gamesPlayedHome,gamesPlayedAway,totalGamesHome,totalWinsHome,homeGamesHome,homeWinsHome,awayGamesHome,awayWinsHome,percentageTotalWinHome,...,statsAway.seasonTotalPossessions,statsAway.last5TotalPossessions,statsAway.seasonTotalPoints,statsAway.last5TotalPoints,statsAway.seasonOffensiveRating,statsAway.last5OffensiveRating,statsAway.seasonDefensiveRating,statsAway.last5DefensiveRating,homeBestOdds,awayBestOdds
594,2017,9,9,8,6,5,4,3,2,0.750,...,736.2,471.4,843,530,114.51,112.43,104.73,106.70,1.55,2.73
721,2017,18,18,20,8,9,4,11,4,0.400,...,1707.0,439.2,2030,538,118.92,122.50,117.46,115.21,2.15,1.81
359,2017,57,55,59,18,27,11,32,7,0.305,...,5195.2,457.6,6205,577,119.44,126.09,112.76,110.58,2.99,1.46
106,2017,36,36,34,17,17,10,17,7,0.500,...,3250.4,469.2,3781,545,116.32,116.16,113.59,110.40,1.22,5.00
89,2017,37,37,35,9,11,3,24,6,0.257,...,3512.2,470.4,4142,529,117.93,112.46,113.86,115.43,2.64,1.60


### Logistic Regression

In [26]:
logreg = LogisticRegression()
logreg.fit(X_train, Y_train)
Y_pred = logreg.predict(X_test)
acc_log = round(logreg.score(X_train, Y_train) * 100, 2)
acc_log

66.260000000000005

In [27]:
coeff_df = pd.DataFrame(games.columns.delete(0))
coeff_df.columns = ['Feature']
coeff_df["Correlation"] = pd.Series(logreg.coef_[0])

coeff_df.sort_values(by='Correlation', ascending=False)

,Feature,Correlation
36,lastGame3AtHomeAway,0.108545
32,lastGame1AtHomeAway,0.103034
11,percentageAwayWinHome,0.090811
9,percentageTotalWinHome,0.086776
10,percentageHomeWinHome,0.081635
34,lastGame2AtHomeAway,0.065074
38,lastGame4AtHomeAway,0.054419
37,lastGame4WinAway,0.050105
28,lastGame4AtHomeHome,0.046599
4,totalWinsHome,0.046143


In [28]:
result = pd.concat([test, pd.DataFrame(Y_pred, columns=['forecast'])], axis=1)
total = len(result)
correct = len(result[(result['homeWin'] == result['forecast'])])
incorrect = len(result[(result['homeWin'] != result['forecast'])])
correct_log = round((correct / total) * 100, 2)

print(correct_log, total, correct, incorrect)

63.6 566 360 206


### Support Vector Machines

In [29]:
svc = SVC()
svc.fit(X_train, Y_train)
Y_pred = svc.predict(X_test)
acc_svc = round(svc.score(X_train, Y_train) * 100, 2)
acc_svc

100.0

In [30]:
result = pd.concat([test, pd.DataFrame(Y_pred, columns=['forecast'])], axis=1)
total = len(result)
correct = len(result[(result['homeWin'] == result['forecast'])])
incorrect = len(result[(result['homeWin'] != result['forecast'])])
correct_svc = round((correct / total) * 100, 2)

print(correct_svc, total, correct, incorrect)

58.13 566 329 237


### k-Nearest Neighbors

In [31]:
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, Y_train)
Y_pred = knn.predict(X_test)
acc_knn = round(knn.score(X_train, Y_train) * 100, 2)
acc_knn

75.609999999999999

In [32]:
result = pd.concat([test, pd.DataFrame(Y_pred, columns=['forecast'])], axis=1)
total = len(result)
correct = len(result[(result['homeWin'] == result['forecast'])])
incorrect = len(result[(result['homeWin'] != result['forecast'])])
correct_knn = round((correct / total) * 100, 2)

print(correct_knn, total, correct, incorrect)

53.18 566 301 265


### Gaussian Naive Bayes

In [33]:
gaussian = GaussianNB()
gaussian.fit(X_train, Y_train)
Y_pred = gaussian.predict(X_test)
acc_gaussian = round(gaussian.score(X_train, Y_train) * 100, 2)
acc_gaussian

64.739999999999995

In [34]:
result = pd.concat([test, pd.DataFrame(Y_pred, columns=['forecast'])], axis=1)
total = len(result)
correct = len(result[(result['homeWin'] == result['forecast'])])
incorrect = len(result[(result['homeWin'] != result['forecast'])])
correct_gaussian = round((correct / total) * 100, 2)

print(correct_gaussian, total, correct, incorrect)

64.13 566 363 203


### Perceptron

In [35]:
perceptron = Perceptron(max_iter = 5)
perceptron.fit(X_train, Y_train)
Y_pred = perceptron.predict(X_test)
acc_perceptron = round(perceptron.score(X_train, Y_train) * 100, 2)
acc_perceptron

46.759999999999998

In [36]:
result = pd.concat([test, pd.DataFrame(Y_pred, columns=['forecast'])], axis=1)
total = len(result)
correct = len(result[(result['homeWin'] == result['forecast'])])
incorrect = len(result[(result['homeWin'] != result['forecast'])])
correct_perceptron = round((correct / total) * 100, 2)

print(correct_perceptron, total, correct, incorrect)

45.58 566 258 308


### Linear SVC

In [37]:
linear_svc = LinearSVC()
linear_svc.fit(X_train, Y_train)
Y_pred = linear_svc.predict(X_test)
acc_linear_svc = round(linear_svc.score(X_train, Y_train) * 100, 2)
acc_linear_svc

63.859999999999999

In [38]:
result = pd.concat([test, pd.DataFrame(Y_pred, columns=['forecast'])], axis=1)
total = len(result)
correct = len(result[(result['homeWin'] == result['forecast'])])
incorrect = len(result[(result['homeWin'] != result['forecast'])])
correct_linear_svc = round((correct / total) * 100, 2)

print(correct_linear_svc, total, correct, incorrect)

56.71 566 321 245


### Stochastic Gradient Descent

In [39]:
sgd = SGDClassifier(max_iter = 5)
sgd.fit(X_train, Y_train)
Y_pred = sgd.predict(X_test)
acc_sgd = round(sgd.score(X_train, Y_train) * 100, 2)
acc_sgd

59.009999999999998

In [40]:
result = pd.concat([test, pd.DataFrame(Y_pred, columns=['forecast'])], axis=1)
total = len(result)
correct = len(result[(result['homeWin'] == result['forecast'])])
incorrect = len(result[(result['homeWin'] != result['forecast'])])
correct_sgd = round((correct / total) * 100, 2)

print(correct_sgd, total, correct, incorrect)

58.13 566 329 237


### Decision Tree

In [41]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, Y_train)
Y_pred = decision_tree.predict(X_test)
acc_decision_tree = round(decision_tree.score(X_train, Y_train) * 100, 2)
acc_decision_tree

100.0

In [42]:
result = pd.concat([test, pd.DataFrame(Y_pred, columns=['forecast'])], axis=1)
total = len(result)
correct = len(result[(result['homeWin'] == result['forecast'])])
incorrect = len(result[(result['homeWin'] != result['forecast'])])
correct_decision_tree = round((correct / total) * 100, 2)

print(correct_decision_tree, total, correct, incorrect)

54.42 566 308 258


### Random Forest

In [43]:
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)
Y_pred = random_forest.predict(X_test)
random_forest.score(X_train, Y_train)
acc_random_forest = round(random_forest.score(X_train, Y_train) * 100, 2)
acc_random_forest

100.0

In [44]:
result = pd.concat([test, pd.DataFrame(Y_pred, columns=['forecast'])], axis=1)
total = len(result)
correct = len(result[(result['homeWin'] == result['forecast'])])
incorrect = len(result[(result['homeWin'] != result['forecast'])])
correct_random_forest = round((correct / total) * 100, 2)

print(correct_random_forest, len(Y_pred), total, correct, incorrect)

62.9 566 566 356 210


## Model Evaluation Results

In [45]:
models = pd.DataFrame({
    'Score': [acc_svc, acc_knn, acc_log, 
              acc_random_forest, acc_gaussian, acc_perceptron, 
              acc_sgd, acc_linear_svc, acc_decision_tree],
    'Correct': [correct_svc, correct_knn, correct_log, 
              correct_random_forest, correct_gaussian, correct_perceptron, 
              correct_sgd, correct_linear_svc, correct_decision_tree],
    'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression', 
              'Random Forest', 'Naive Bayes', 'Perceptron', 
              'Stochastic Gradient Decent', 'Linear SVC', 
              'Decision Tree']})
models.sort_values(by='Correct', ascending=False)

,Correct,Model,Score
4,64.13,Naive Bayes,64.74
2,63.60,Logistic Regression,66.26
3,62.90,Random Forest,100.00
0,58.13,Support Vector Machines,100.00
6,58.13,Stochastic Gradient Decent,59.01
7,56.71,Linear SVC,63.86
8,54.42,Decision Tree,100.00
1,53.18,KNN,75.61
5,45.58,Perceptron,46.76


#### Sample of ten incorrect predictions

In [46]:
result[(result['homeWin'] != result['forecast'])].sample(10)

,id,season,gamesPlayedHome,gamesPlayedAway,totalGamesHome,totalWinsHome,homeGamesHome,homeWinsHome,awayGamesHome,awayWinsHome,...,statsAway.seasonTotalPoints,statsAway.last5TotalPoints,statsAway.seasonOffensiveRating,statsAway.last5OffensiveRating,statsAway.seasonDefensiveRating,statsAway.last5DefensiveRating,homeWin,homeBestOdds,awayBestOdds,forecast
426,42693,2017,9,9,8,4,4,2,4,2,...,878,578,117.38,125.22,116.98,116.55,True,3.20,1.53,False
60,42159,2017,37,36,35,18,16,11,19,7,...,3569,522,116.37,114.22,110.76,104.81,False,1.22,5.70,True
427,42694,2017,11,10,9,4,5,3,4,1,...,1094,537,120.27,114.21,118.07,114.21,True,1.28,4.46,False
167,42284,2017,61,46,60,41,31,20,29,21,...,4901,612,118.78,120.85,116.33,112.36,True,3.56,1.38,False
215,42340,2017,49,48,49,27,23,14,26,13,...,5160,542,117.44,120.34,113.92,117.45,True,2.09,1.87,False
62,42162,2017,37,37,35,9,11,3,24,6,...,4142,529,117.93,112.46,113.86,115.43,True,2.64,1.60,False
324,42468,2017,58,54,56,20,26,13,30,7,...,5757,561,115.19,113.52,110.68,113.11,True,1.14,7.20,False
522,42817,2017,18,18,18,11,9,6,9,5,...,1803,541,117.52,122.95,114.78,108.41,False,1.41,3.24,True
222,42348,2017,48,48,48,28,24,14,24,14,...,4964,519,114.46,111.61,112.76,105.81,False,1.56,2.70,True
466,42749,2017,14,15,13,4,6,2,7,2,...,1562,587,116.64,122.91,117.46,119.77,True,1.39,3.36,False


## Calculate Profit

#### Number of games with predictions

In [47]:
len(result)

566

### Iterate through all predicted games
* Gamble $50 on all games and calculate result

In [48]:
# Create a dataframe for the results - same size as dataset
data = pd.DataFrame(index=range(0,len(result)), columns=['winnings'])

bank = 0;
betAmount = 50

for index, game in result.iterrows():
    winnings = 0
    if (game.forecast == True) & (game.homeWin == True):
        winnings = (game.homeBestOdds * betAmount);
    if (game.forecast == False) & (game.homeWin == False):
        winnings = (game.awayBestOdds * betAmount)
    if (game.forecast != game.homeWin):
        winnings = 0 - betAmount

    data.loc[index] = [winnings]
    bank += winnings

result = pd.merge(result, data, left_index=True, right_index=True)
    
bank

19830.5

In [49]:
result[['id', 'winnings', 'homeWin', 'forecast', 'season', 'homeBestOdds', 'awayBestOdds', 'gamesPlayedHome', 'gamesPlayedAway', 'totalGamesHome', 'totalWinsHome', 'homeGamesHome', 'homeWinsHome']].sample(30)

,id,winnings,homeWin,forecast,season,homeBestOdds,awayBestOdds,gamesPlayedHome,gamesPlayedAway,totalGamesHome,totalWinsHome,homeGamesHome,homeWinsHome
205,42328,69,True,True,2017,1.38,3.50,49,48,47,27,22,19
359,42512,-50,True,False,2017,1.46,3.00,60,31,59,17,29,12
487,42774,-50,False,True,2017,1.50,2.85,16,16,15,8,8,6
321,42464,179,True,True,2017,3.58,1.35,59,54,59,46,30,23
523,42818,55.5,True,True,2017,1.11,8.80,18,19,16,6,9,5
22,42112,69.5,True,True,2017,1.39,3.27,34,33,32,19,16,10
488,42775,-50,False,True,2017,1.33,3.82,16,13,15,7,8,3
87,42190,71.5,True,True,2017,1.43,3.22,38,39,35,20,19,13
369,42524,-50,False,True,2017,2.09,1.91,60,30,59,19,31,11
485,42772,91.5,True,True,2017,1.83,2.14,13,16,8,6,5,4


In [50]:
result['winnings'].sum()

19830.5

In [51]:
if save_result_data == True:
    result.to_csv(output_result_data_filename, encoding='utf-8')

In [52]:
#result[(result['forecast'] == False) & (result['oddsBet365Home'] < result['oddsBet365Away'])]

#### Test: Bet on the Home team regardless

In [53]:
bank = 0;
betAmount = 50

for index, game in result.iterrows():
    winnings = 0
    if (game.homeWin == True):
        winnings = (game.homeBestOdds * betAmount);
    else:
        winnings = 0 - betAmount
        
    bank += winnings
    
bank

17514.5

#### Test: Bet on the Away team regardless

In [54]:
bank = 0;
betAmount = 50

for index, game in result.iterrows():
    winnings = 0
    if (game.homeWin == False):
        winnings = (game.awayBestOdds * betAmount);
    else:
        winnings = 0 - betAmount
        
    bank += winnings
    
bank

14976.0

#### Test: Bet on the team with the lowest odds

In [55]:
bank = 0;
betAmount = 50

for index, game in result.iterrows():
    winnings = 0
    if (game.homeWin == False) & (game.awayBestOdds < game.homeBestOdds):
        winnings = (game.awayBestOdds * betAmount);
    if (game.homeWin == True) & (game.homeBestOdds < game.awayBestOdds):
        winnings = (game.homeBestOdds * betAmount);
    if winnings == 0:
        winnings = 0 - betAmount
        
    bank += winnings
    
bank

15211.5